In [1]:
# installations
!pip install numpy
!pip install rasterio
!pip install pykrige
!apt install gdal-bin python-gdal python3-gdal 
!apt install python3-rtree 
!pip install git+git://github.com/geopandas/geopandas.git
!pip install descartes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
The following package was automatically installed and is no longer required:
  lib

In [4]:
# imports
import numpy as np
import rasterio as rio
import pandas as pd
import math
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
import matplotlib.pyplot as plt
import random

opening file

In [5]:
with rio.open('dem.tif') as file:
  dataset = file.read(1)
  row , column = dataset.shape

In [ ]:
print(column , row)
903/20


903 566


45.15

take random values

In [20]:
# pixelx = random.sample(range(0,dataset.shape[1]) , 100)
pixelx = [int(i) for i in list(np.linspace(0 , column-1 ,700)) ]
# pixely = random.sample(range(0,dataset.shape[0]) , 100)
pixely = [int(i) for i in list(np.linspace(0 , row-1 , 700)) ]
data = np.zeros((dataset.shape))
# data[pixely][pixelx] = dataset[pixely][pixelx]
value = []
for x,y in zip(pixelx,pixely):
  index_value = dataset[y][x]
  data[y][x] = index_value
  value.append(index_value)

In [ ]:
print(len(pixelx))
print(len(pixely))

40
40


In [7]:
def find_neighbor(num , pixelx_temp , pixely_temp , x1 , y1):
  near_x = sorted(pixelx, key=lambda x:abs(x-x1))[:3]
  near_y = sorted(pixely, key=lambda x:abs(x-y1))[:3]
  return near_x , near_y



IDW

In [8]:
# distance of two pixels
def gamma(x1,x2,y1,y2):
  return np.sqrt(((x1-x2)**2) + ((y1-y2)**2))

In [ ]:
# for each pixel in data
for x in range(0 , dataset.shape[1]):
  for y in range(0 ,dataset.shape[0] ):
    # if the pixel is zero meaning it's not bench mark
    if data[y][x] == 0:
      # weight of each BM for the pixel
      weight = []
      value = []
      neighbors = find_neighbor(3,pixelx , pixely , x , y)
      for x1 in neighbors[0]:
        for y1 in neighbors[1]:
            index_weight = 1/gamma(x1,x,y1,y)
            weight.append(index_weight)
            value.append(dataset[y1][x1])
          # get the value calculated for pixel by formula
      data[y][x] = (sum(list(np.multiply(weight,value))))/(sum(weight))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


In [ ]:
create_tif(data , dataset , file , 'dem_IDW_local_40_not_delete1')

Kriging (manual)

In [57]:
def station_gammas(pixelx , pixely):
  # make a len(coordinates)*len(coordinates) length array for distances of stations
  distance=[]
  for x1 in pixelx:
    for y1 in pixely:
      for x2 in pixelx:
        for y2 in pixely:
          distance.append(gamma(x1,y1,x2,y2))

  # for x1,y1 in zip(pixelx,pixely):
  #   for x2,y2 in zip(pixelx,pixely):
  #     distance.append(gamma(x1,y1,x2,y2))

  # reshape distance of stations array to 34*34 2D array
  distance = np.reshape(distance , (len(pixelx)*len(pixelx),len(pixely)*len(pixely)))

  # add a 1*len(coordinates) row to the end of the distance array
  station_gamma = np.vstack((distance,np.ones((1,len(pixelx)*len(pixelx)))))

  # add a len(coordinates)+1*1 column to the end of the distance array
  station_gamma = np.hstack((station_gamma,np.ones((len(pixelx)*len(pixelx)+1,1))))

  # the last cell of the array should be zero
  station_gamma[-1,-1]=0

  try:
    station_gamma_inverse = np.linalg.inv(station_gamma)
    return station_gamma_inverse
  except:
    print(np.linalg.matrix_rank(station_gamma))

  

In [58]:

for y in range(row):
  for x in range(column):
    if data[y,x]==0:
      sample_distance=[]
      value = []
      neighbors = find_neighbor(3,pixelx , pixely , x , y)
      # print(neighbors)
      neighbors_gamma_inverse = station_gammas(neighbors[0] , neighbors[1])
      # print(neighbors_gamma_inverse.shape)
      for stationx in neighbors[0]:
        for stationy in neighbors[1]:
          sample_distance.append(gamma(stationx,stationy,x,y))
          value.append(dataset[stationy][stationx])
      # for x,y in zip(neighbors[0],neighbors[1]):
      #   sample_distance.append(gamma(stationx,stationy,x,y))
      sample_distance.append(1)
      sample_distance = np.array(sample_distance).T
      # print(len(sample_distance))
      
      weight0 = np.dot(neighbors_gamma_inverse,sample_distance)
      weight = weight0[0:-1]
      

      Z = sum(np.multiply(weight,value))
      data[y][x] = Z



7


TypeError: ignored

In [49]:
create_tif(data , dataset , file , 'dem_Kriging_manual_40_not')

IndexError: ignored

Kriging(pykrige)

In [8]:
def UK_model (pixelx , pixely , value ,row ,column ): 

    # making grid
    gridx = np.arange(0.0, column, 1)
    gridy = np.arange(0.0, row, 1)
    # x = []
    # y = []
    # val = []
    # for x1 in pixelx:
    #   for y1 in pixely:
    #     x.append(x1)
    #     y.append(y1)
    #     val.append(dataset[y1][x1])

    UK = UniversalKriging(
      # data['pixelx'],
      pixelx,
      # data['pixely'],
      pixely,
      # data[Name_pollution],
      value,
      variogram_model="power",
      nlags = 3,
      weight= False,
      anisotropy_scaling = 1,
      anisotropy_angle = 0,
      # drift_terms=["regional_linear"],
      exact_values = True,
      pseudo_inv = False,
      # pseudo_inv_type = 'pinv'
      )
    
    z, ss = UK.execute("grid", gridx, gridy)
    return z , ss

In [ ]:
z , ss = UK_model(pixelx , pixely , value , row , column)

In [19]:
create_tif(z , dataset , file , 'dem_pykrige_universal_900')

In [ ]:
def validation_UK_Model (pixelx , pixely , value ,row ,column ):
      #create column of prediction
      data['prediction'] = np.zeros(len(data))

      for i in range(len(data)):
          # delete value we want to predict
          model_data = data.drop(i)

          #predict values
          model_values , model_variances = UK_model(data = model_data ,row = row ,column = column)

          #add predict value for deleted coordination
          data['prediction'][i] = model_values[ data['pixely'][i] , data['pixelx'][i]]
      
      return data

In [ ]:
def OK_model (data ,row ,column): 

    # making grid
    gridx = np.arange(0.0, column, 1)
    gridy = np.arange(0.0, row, 1)

    OK = OrdinaryKriging(
     # data['pixelx'],
      pixelx,
      # data['pixely'],
      pixely,
      # data[Name_pollution],
      value,
      variogram_model="power",
      nlags = 2,
      anisotropy_scaling = 1,
      anisotropy_angle = 0,
      enable_statistics = True,
      exact_values = True,
      pseudo_inv = False,
      # pseudo_inv_type = 'pinvh',
      weight= False,
      )
    
    z, ss = OK.execute("grid", gridx, gridy)
    return z , ss

In [ ]:
z , ss = OK_model(pixelx , pixely , value , row , column)

In [ ]:
def validation_OK_Model (data ,row ,column,Name_pollution):
      #create column of prediction
      data['prediction'] = np.zeros(len(data))

      for i in range(len(data)):
          # delete value we want to predict
          model_data = data.drop(i)

          #predict values
          model_values , model_variances = OK_model(data = model_data ,row = row ,column = column,Name_pollution=Name_pollution)

          #add predict value for deleted coordination
          data['prediction'][i] = model_values[ data['pixely'][i] , data['pixelx'][i]]
      
      return data

making Tif from data

In [11]:
def create_tif(data , dataset , file , name):
  new_dataset1 = rio.open(f'{name}.tif',
                    'w', 
                    driver='GTiff',
                    height = data.shape[0], 
                    width = data.shape[1],
                    count=1,
                    dtype=dataset.dtype,
                    crs='+proj=latlong',
                    transform=file.transform,
                    )
  new_dataset1.write(data, 1)
  new_dataset1.close()

In [7]:
def RMSE_cal(predictions , targets):
  return np.sqrt(np.mean((predictions-targets)**2))